In [2]:
pip install pandas

In [3]:
pip install yfinance

In [4]:
pip install statsmodels

In [5]:
pip install matplotlib

In [6]:
pip install cufflinks

In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [8]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns # Import Seaborn

# Set the Seaborn style using Seaborn's API
sns.set_theme(style="darkgrid") # Apply the 'darkgrid' theme

# Continue with your Matplotlib customizations
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

#Multi-Factor Models

##Implementing the CAPM in Python

Specify the risky asset and the time horizon:

In [10]:
RISKY_ASSET = 'BBRI.JK'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2020-01-01'
END_DATE = '2023-12-31'

Download data from Yahoo Finance:

In [11]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                start=START_DATE,
                end=END_DATE,
                # remove 'adjusted=True', and use 'auto_adjust=True' instead if needed
                progress=False,
                auto_adjust=True # if you need adjusted closing prices
                )

print(f'Downloaded {df.shape[0]} rows of data.')

Downloaded 1034 rows of data.


Resample to monthly data and calculate simple returns:

In [12]:
X = df['Close'].rename(columns={RISKY_ASSET: 'asset',
                                  MARKET_BENCHMARK: 'market'}) \
                  .resample('M') \
                  .last() \
                  .pct_change() \
                  .dropna()

X.head()

Ticker,asset,market
Date,,
2020-02-29,-0.023890,-0.084110
2020-03-31,-0.279236,-0.125119
2020-04-30,-0.096026,0.126844
2020-05-31,0.080586,0.045282
2020-06-30,0.027119,0.018388


Calculate beta using the covariance approach:

In [13]:
covariance = X.cov().iloc[0,1]
benchmark_variance = X.market.var()
beta = covariance / benchmark_variance
beta

0.55674335775219

Prepare the input and estimate CAPM as a linear regression:

In [14]:
# import the necessary module from statsmodels
import statsmodels.api as sm

In [19]:
# separate target
y = X.pop('asset')

# add constant
X = sm.add_constant(X)

# define and fit the regression model
capm_model = sm.OLS(y, X).fit()

# print results
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     7.750
Date:                Wed, 27 Nov 2024   Prob (F-statistic):            0.00783
Time:                        14:37:41   Log-Likelihood:                 54.321
No. Observations:                  47   AIC:                            -104.6
Df Residuals:                      45   BIC:                            -100.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0088      0.012      0.766      0.4